## INITIALIZATION

In [1]:
%matplotlib notebook
import numpy as np
import time
import tables as tb
import matplotlib.pyplot as plt

from tjmonopix.tjmonopix import TJMonoPix
from tjmonopix.online_monitor import noise_monitor, plots
from tjmonopix.analysis import analysis_functions

import tjmonopix.scans.injection_scan

chip = TJMonoPix(conf="../tjmonopix/tjmonopix_mio3.yaml", no_power_reset=False)
chip.init(fl="EN_HV")

chip['data_rx'].CONF_START_FREEZE = 64 #default 3
chip['data_rx'].CONF_STOP_FREEZE = 100 #default 40
chip['data_rx'].CONF_START_READ = 66 #default 6
chip['data_rx'].CONF_STOP_READ = 68 #default 7
chip['data_rx'].CONF_STOP = 105 #default 45


2022-02-09 12:28:07,996 [INFO ] (MainThread) GPAC: power reset
2022-02-09 12:28:09,730 [INFO ] (MainThread) Found adapter card: General Purpose Analog Card (GPAC) with ID 3
2022-02-09 12:28:09,963 [INFO ] (MainThread) Found board 0 running firmware version 0.0
2022-02-09 12:28:10,200 [INFO ] (MainThread) ibias = 45
2022-02-09 12:28:10,217 [INFO ] (MainThread) ibias = 503.125 nA
2022-02-09 12:28:10,786 [INFO ] (MainThread) {'VPC [V]': 1.2998506463536463, 'VDDA_DAC [mA]': 2.502471747485684, 'VDDP [V]': 1.8049802771084336, 'VDDP OC': False, 'VDDA [mA]': 50.151368865825674, 'VDDD [V]': 1.8015140363636364, 'VDDA_DAC OC': False, 'VDDD [mA]': 2.820288999038665e-08, 'VPCSWSF [V]': 0.5001542395802099, 'VDDP [mA]': 128.39714805315754, 'BiasSF [V]': 1.062780289971045, 'VDDD OC': False, 'VPC [mA]': 0.1364994039805877, 'VPCSWSF [mA]': 0.19709780513238037, 'VDDA [V]': 1.8058565495495498, 'VDDA OC': False, 'VDDA_DAC [V]': 1.8035484515484517, 'BiasSF [mA]': 0.10059447096451782}


## POWER STATUS

In [2]:
#chip['BiasSF'].set_current(100, unit='uA') # SET THE SOURCE FOLLOWER READOUT CURRENT
chip.get_power_status()

{'BiasSF [V]': 1.0632804013424586,
 'BiasSF [mA]': 0.10059447096451782,
 'VDDA OC': False,
 'VDDA [V]': 1.8058565495495498,
 'VDDA [mA]': 50.151368865825674,
 'VDDA_DAC OC': False,
 'VDDA_DAC [V]': 1.8045472377622378,
 'VDDA_DAC [mA]': 3.002966091804109,
 'VDDD OC': False,
 'VDDD [V]': 1.8015140363636364,
 'VDDD [mA]': 2.820288999038665e-08,
 'VDDP OC': False,
 'VDDP [V]': 1.8049802771084336,
 'VDDP [mA]': 128.39714805315754,
 'VPC [V]': 1.2993515804695306,
 'VPC [mA]': -0.2749987832365381,
 'VPCSWSF [V]': 0.5001542395802099,
 'VPCSWSF [mA]': -0.11455683661374243}

In [ ]:
"""
# SELECT WHICH DOUBLE COLUMNS (PIXEL "FLAVOURS") TO ENABLE
chip['CONF_SR']['EN_PMOS_NOSF'].setall(False)
chip['CONF_SR']['EN_PMOS'].setall(False)
chip['CONF_SR']['EN_COMP'].setall(False)
chip['CONF_SR']['EN_HV'].setall(True)
chip['CONF_SR']['EN_OUT'].setall(False) #ENABLES OR DISABLES THE NORMAL OUTPUT PADS, ACTIVE LOW
chip['CONF_SR']['nEN_OUT'].setall(True) #ENABLES OR DISABLES THE COMPLEMENTARY OUTPUT PADS, ACTIVE LOW
chip['CONF_SR']['EN_HITOR_OUT'].setall(True) #ENABLES OR DISABLES THE NORMAL HITOR PADS, HITOR0-3 =  1-4 flavor, ACTIVE LOW
chip['CONF_SR']['nEN_HITOR_OUT'].setall(True) #ENABLES OR DISABLES THE COMPLEMENTARY HITOR PADS, ACTIVE LOW
chip.write_conf()

# SELECT WHICH PHYSICAL COLUMS TO INJECT
# INJ_IN_MON_L AND INJ_IN_MON_L SELECT THE LEFT AND RIGHT SPECIAL ANALOG MONITORING PIXELS
chip['CONF_SR']['COL_PULSE_SEL'].setall(False)

chip['CONF_SR']['INJ_IN_MON_L'] = 1 # ENABLE INJECTION FOR THE ANALOG MONITORING PIXELS LEFT SIDE
chip['CONF_SR']['INJ_IN_MON_R'] = 1 # ENABLE INJECTION FOR THE ANALOG MONITORING PIXELS RIGHT SIDE

# SELECT WHICH PHYSICAL ROWS TO INJECT
# THE SPEXIAL PIXELS OUTA_MON3 to OUTA_MON0 CORRESPONT TO ROWS 223 to 220 FOR INJECTION
chip['CONF_SR']['INJ_ROW'].setall(False)
chip['CONF_SR']['INJ_ROW'][223:220] = True # FOR THE ANALOG MONITORING TOP PIXEL

# SELECT PHYSICAL COLUMNS AND ROWS FOR INJECTION WITH FUNCTION
#chip.enable_injection(3,18,99)
chip.write_conf()


# SELECT PHYSICAL COLUMN(S) FOR HITOR OUTPUT
# nMASKH (SO SETTING MASKH TO FALSE) ENABLES HITOR FOR THE SPECIFIC ROW
chip['CONF_SR']['DIG_MON_SEL'].setall(False)
#chip.enable_column_hitor(1,18)

chip.write_conf()
"""

## ANALOG FRONT END CONFIGURATION

In [3]:
vl_dac = 40
vh_dac = 80
vreset_dac = 35
icasn_dac = 0  # This sets the baseline voltage of the discriminator input
ireset_dac = 2
ithr_dac = 5  # Also this sets the baseline voltage of the discriminator input
idb_dac = 50  # This sets the threshold of the discriminator. It is changed later.
ibias_dac = 100

## SET THE INJECTION PULSE AMPLITUDE
## 128-bit DAC (7-bit binary equivalent)
## SET THE VOLTAGES IN ONE HOT ENCODING, ONLY ONE BIT ACTIVE AT A TIME.
## V = (127/1.8)*#BIT
# The default values are VL=44, VH=79, VH-VL=35
# VDAC LSB=14.17mV, Cinj=230aF, 1.43e-/mV, ~710e-  
CAPCAL=20.3439450687
vl = chip.set_vl_dacunits(vl_dac,1)
vh = chip.set_vh_dacunits(vh_dac,1)

chip.write_conf()

# SET VRESET_P, THIS IS THE BASELINE OF THE FRONT END INPUT, ONE HOT ENCODING
vreset = chip.set_vreset_dacunits(vreset_dac, 1) #1V

## 128-bit DAC (7-bit binary equivalent)
## SET THE CURRENTS USING THERMOMETER ENCODING, I = #BITS_ACTIVE*140nA*SCALING, SCALING IS DIFFERENT FOR EACH CURRENT
## SCALING: IBIAS=10, IDB=16, ITHR=0.125, ICASN=4, IRESET=0.03125
## ACTIVE BITS SHOULD BE SET STARTING FROM THE MIDDLE e.g. for 15 active bits, (128-15)/2=56,5 so 56zeros,15ones,57zeros
## Thus, Ix[71:57] = True

# SET ICASN, THIS CURRENT CONTROLS THE OUTPUT BASELINE, BE CAREFUL NOT TO SET IT TO HIGH
# ALWAYS MONITOR THE POWER AFTER SETTING ICASN. IF VDDD IS SEVERAL mA, REDUCE IT UNTIL IT RETURNS TO 0
# ICASN MAINLY CONTROLS THE THRESHOLD
icasn = chip.set_icasn_dacunits(icasn_dac,1) #4.375nA # approx 1.084V at -3V backbias, 600mV at 0V backbias

# SET IRESET, THIS CURRENT CONTROLS THE RESET RATE OF THE FRONT END INPUT (ALSO THE THRESHOLD)
ireset = chip.set_ireset_dacunits(ireset_dac,1,1) #270pA, HIGH LEAKAGE MODE, NORMAL SCALING, 0 = LOW LEAKAGE MODE, SCALING*0.01

# SET ITHR, THIS CURRENT CONTROLS THE RESET RATE OF THE OUTPUT (AND THE THRESHOLD)
ithr = chip.set_ithr_dacunits(ithr_dac,1) #680pA ## default=5

# SET ITHR, THIS CURRENT CONTROLS THE BIASING OF THE DISCRIMINATOR (AND THE THRESHOLD)
idb = chip.set_idb_dacunits(idb_dac,1) #500nA  

# SET IBIAS, THIS CURRENT IS THE DC CURRENT OF THE MAIN BRANCH OF THE FRONT END THAT PROVIDES AMPLIFICATION
# IT CONTROLS MAINLY THE RISE TIME
ibias = chip.set_ibias_dacunits(ibias_dac,1) #500nA default=50

############ ENABLE THE DAC CURRENT MONITORING ###########
# chip['CONF_SR']['SWCNTL_DACMONI'] = 0

########## SET THE BIAS CURRENTS OF THE TWO STAGE SOURCE FOLLOWER THAT BUFFERS THE ANALOG MONITORING VOLTAGES #########
# CONTROLS THE RESPONSE TIME AND THE LEVEL SHIFT OF THE BUFFER
# chip['CONF_SR']['SET_IBUFN_L'] = 0b1001
# chip['CONF_SR']['SET_IBUFP_L'] = 0b0101

chip.write_conf()


setted_param_dac = np.array([vl_dac, vh_dac, vreset_dac, icasn_dac, ireset_dac, ithr_dac, idb_dac, ibias_dac])
setted_param = np.array([vl, vh, vreset, icasn, ireset, ithr, idb, ibias])

2022-02-09 12:28:22,671 [INFO ] (MainThread) vl = 0.951929133858V
2022-02-09 12:28:22,686 [INFO ] (MainThread) vh = 1.51885826772V
2022-02-09 12:28:22,701 [INFO ] (MainThread) vreset = 1.05106299213V
2022-02-09 12:28:22,717 [INFO ] (MainThread) icasn = 0
2022-02-09 12:28:22,733 [INFO ] (MainThread) icasn = 4.375 nA
2022-02-09 12:28:22,739 [INFO ] (MainThread) ireset = 2 high leakage mode
2022-02-09 12:28:22,755 [INFO ] (MainThread) ireset = 0.1025390625nA, high leakage mode
2022-02-09 12:28:22,770 [INFO ] (MainThread) ithr = 5
2022-02-09 12:28:22,786 [INFO ] (MainThread) ithr = 0.8203125 nA
2022-02-09 12:28:22,802 [INFO ] (MainThread) idb = 50
2022-02-09 12:28:22,818 [INFO ] (MainThread) idb = 892.5 nA
2022-02-09 12:28:22,832 [INFO ] (MainThread) ibias = 100
2022-02-09 12:28:22,855 [INFO ] (MainThread) ibias = 1104.6875 nA


## ENABLE MASK AND GET OCCUPANCY

In [4]:
th = 2
step = 50
dt = 0.2

noisy_pixels, n_disabled_pixels, mask = chip.auto_mask(th=th, step=step, dt=dt)
print("========= Getting residual occupancy =============")
pix_tmp, cnt = chip.get_occupancy(dt);

Enable MASKH 50
Got 51222 hits, total noisy pixels: 33
Enable MASKH 100
Got 105850 hits, total noisy pixels: 93
Enable MASKH 150
Got 26401 hits, total noisy pixels: 143
Enable MASKH 200
Got 154760 hits, total noisy pixels: 209
Enable MASKH 223
Got 36621 hits, total noisy pixels: 232
Enable MASKV 50
Got 10 hits, total noisy pixels: 234
Enable MASKV 100
Got 14 hits, total noisy pixels: 237
Enable MASKV 111
Got 82 hits, total noisy pixels: 239
Enable MASKD 50
Got 6 hits, total noisy pixels: 239
Enable MASKD 100
Got 77 hits, total noisy pixels: 240
Enable MASKD 150
Got 7 hits, total noisy pixels: 240
Enable MASKD 200
Got 85 hits, total noisy pixels: 243
Enable MASKD 250
Got 54 hits, total noisy pixels: 244
Enable MASKD 300
Got 119 hits, total noisy pixels: 246
Enable MASKD 350
Got 127 hits, total noisy pixels: 247
Enable MASKD 400
Got 54 hits, total noisy pixels: 249
Enable MASKD 447
Got 24 hits, total noisy pixels: 252
Checking again after masking
Got 126 hits, total noisy pixels: 252
Noi

# Configure injection and Select 1pixel 

In [ ]:
import tjmonopix.scans.injection_scan
reload(tjmonopix.scans.injection_scan)

delay = 800 #in clock units (640 MHz)
width = 250
repeat = 100 #number of pulse injected
chip['inj'].set_delay(delay)
chip['inj'].set_width(width)
chip['inj'].set_repeat(repeat)
chip['inj'].set_phase(0)
chip['inj'].set_en(0)

idb = chip.set_idb_dacunits(50,1) #### threshold; SOPRA LA METTE CON 50 DAC
vh = chip.set_vh_dacunits(100,1) #### injection; SOPRA LA METTE CON 80 DAC
chip.write_conf()

col_to_inject = 66
row_to_inject = 10
chip.select_injection(col_to_inject, row_to_inject)

_, pixels, hits_per_pixel = plots.tj_plot(chip, wait_inj=True)
chip['data_rx'].set_en(False)

chip.set_monoread()
for _ in range(5):
    chip['fifo'].reset()
    time.sleep(0.002)
chip["inj"].start()
while not chip['inj'].is_ready:
    time.sleep(0.001)
time.sleep(0.2)

print "fifo size", chip['fifo'].get_fifo_size()
ix = chip.interpret_data(chip['fifo'].get_data())
uni, cnt = np.unique(ix[["col","row"]], return_counts = True)
arg = np.argsort(cnt)
for a_i, a in enumerate(arg[::-1]):
    print a_i, uni[a], cnt[a]
    

## run parameter space scan using injection_scan

In [5]:
chip['CONF_SR']['INJ_ROW'].setall(False)
chip['CONF_SR']['COL_PULSE_SEL'].setall(False)

delay = 800 #in clock units (640 MHz)
width = 250
repeat = 100 #number of pulse injected
chip['inj'].set_delay(delay)
chip['inj'].set_width(width)
chip['inj'].set_repeat(repeat)
chip['inj'].set_phase(0)
chip['inj'].set_en(0)

idb = chip.set_idb_dacunits(20,1) 
vh = chip.set_vh_dacunits(80,1) # The functions below change vl w.r.t. vh
chip.write_conf()

scans = tjmonopix.scans.injection_scan.InjectionScan(dut=chip)

collist = [90]
rowlist = [70] #list(range(224))
injlist = np.arange(50, 0, -1)
thlist = [20]
chip.cleanup_fifo(n=10)
chip.cleanup_fifo(n=10)

output_filename = scans.start(
    collist = collist, rowlist = rowlist,
    injlist = injlist, thlist = thlist, phaselist = None,
    with_mon = False, n_mask_col = 1, debug = 4)
chip.cleanup_fifo(n=10)

scans.analyze(output_filename)

2022-02-09 12:28:48,298 [INFO ] (MainThread) idb = 20
2022-02-09 12:28:48,313 [INFO ] (MainThread) idb = 367.5 nA
2022-02-09 12:28:48,328 [INFO ] (MainThread) vh = 1.51885826772V
2022-02-09 12:28:48,351 [INFO ] (MainThread) Initializing InjectionScan
2022-02-09 12:28:54,098 [INFO ] (MainThread) Power status: {'VPC [V]': 1.2993515804695306, 'VDDA_DAC [mA]': 2.502471747485684, 'VDDP [V]': 1.8049802771084336, 'VDDP OC': False, 'VDDA [mA]': 79.53297889485958, 'VDDD [V]': 1.8015140363636364, 'VDDA_DAC OC': False, 'VDDD [mA]': 2.820288999038665e-08, 'VPCSWSF [V]': 0.5001542395802099, 'VDDP [mA]': 143.50269488026495, 'BiasSF [V]': 1.0652808468281127, 'VDDD OC': False, 'VPC [mA]': 0.17799922116044364, 'VPCSWSF [mA]': 0.19709780513238037, 'VDDA [V]': 1.8038584294294295, 'VDDA OC': False, 'VDDA_DAC [V]': 1.8045472377622378, 'BiasSF [mA]': 0.10059447096451782}
2022-02-09 12:29:11,371 [INFO ] (MainThread) Temperature (C): 30.678964
2022-02-09 12:29:11,757 [INFO ] (MainThread) Temperature (C): 30.5

sleeping


2022-02-09 12:29:17,950 [WARNI] (MainThread) SRAM FIFO not empty after reset: size = 2809856
2022-02-09 12:29:18,582 [INFO ] (MainThread) Set timestamp: src=inj
2022-02-09 12:29:19,207 [INFO ] (MainThread) Starting FIFO readout
2022-02-09 12:29:19,221 [WARNI] (MainThread) SRAM FIFO not empty when starting FIFO readout: size = 345448


('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)


2022-02-09 12:29:19,523 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:19,553 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:19,601 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:19,638 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected


...

2022-02-09 12:29:20,286 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,325 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,371 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,410 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,440 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,487 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected


('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)


2022-02-09 12:29:20,526 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,572 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,604 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,641 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,673 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,703 [INFO ] (MainThread) scan_param_id=0 dat=3617114: cols=[90]


('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)
('row in rowlist', 70)


2022-02-09 12:29:20,703 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,773 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,811 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,842 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,888 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,927 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,957 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:20,989 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:21,026 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:21,073 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:21,105 [ERROR] (WatchdogThread) DATA RX FIFO discard error(s) detected
2022-02-09 12:29:21,142 [ERROR] 